In [1]:
from numpy.ma.core import repeat

from Launch import  Scheduler
import unittest
from datetime import datetime, timezone, timedelta

import time
from unittest.mock import patch, MagicMock
from threading import Event


In [2]:
# როდესაც ჯობიისთის გადაცემული გაშვების თარიღი

### Intitalisation Test

In [3]:
class TestSchedulerInitialization(unittest.TestCase):
    def test_scheduler_initialization(self):
        """Test the initialization of the Scheduler class."""
        
        scstart = "2025-12-25 10:00"
        tmzone = 5
    
        local_timezone = datetime.now().astimezone().tzinfo
        # Parse the original date
        original_datetime = datetime.strptime(scstart, "%Y-%m-%d %H:%M")
    
        # Add the original timezone dynamically
        original_timezone = timezone(timedelta(hours=tmzone))
        original_datetime = original_datetime.replace(tzinfo=original_timezone)
    
        scheduler = Scheduler(threading=False, start_date=scstart, time_zone=tmzone)
    
        if scstart:
            if local_timezone != original_timezone:
                comparison = original_datetime.astimezone(local_timezone)
                self.assertEqual(scheduler.scheduler_startdate, comparison)
            else:
                self.assertEqual(scheduler.scheduler_startdate, scstart)
        else:
            print('No start date')


if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(TestSchedulerInitialization)
    unittest.TextTestRunner(verbosity=2).run(suite)



test_scheduler_initialization (__main__.TestSchedulerInitialization.test_scheduler_initialization)
Test the initialization of the Scheduler class. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


### Job Creation Testing

In [4]:
class TestSchedulerJobCreation(unittest.TestCase):

    def test_job_creation(self):
        """Test creating a job within the scheduler."""
        timezonesc = 5
        timezonejob = 0
        local_timezone = datetime.now().astimezone().tzinfo

        start_datesc = "2025-12-25 10:00"
        startdatejob = "2025-12-25 11:00"
        
        scheduler = Scheduler(start_date=start_datesc, time_zone=timezonesc)
        job_to_run = scheduler.job(start_date=startdatejob, time_zone=timezonejob)

        if timezonejob is not None:
            original_timezone = timezone(timedelta(hours=timezonejob))
            if local_timezone != timezonejob:
                original_datetime = datetime.strptime(startdatejob, "%Y-%m-%d %H:%M")
                original_datetime = original_datetime.replace(tzinfo=original_timezone)
                comparison = original_datetime.astimezone(local_timezone)
                self.assertEqual(job_to_run.job['startdate'], comparison)
            else:
                self.assertEqual(job_to_run.job['startdate'], startdatejob)
        elif timezonejob is None and timezonesc is not None:
            original_timezone = timezone(timedelta(hours=timezonesc))
            if timezonesc != local_timezone:
                original_datetime = datetime.strptime(startdatejob, "%Y-%m-%d %H:%M")
                original_datetime = original_datetime.replace(tzinfo=original_timezone)
                comparison = original_datetime.astimezone(local_timezone)
                self.assertEqual(job_to_run.job['startdate'], comparison)
            else:
                self.assertEqual(job_to_run.job['startdate'], startdatejob)

        else:
            print('??????')

if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(TestSchedulerJobCreation)
    unittest.TextTestRunner(verbosity=2).run(suite)


test_job_creation (__main__.TestSchedulerJobCreation.test_job_creation)
Test creating a job within the scheduler. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


### Timezone Testing All

In [5]:
class TestSchedulerTimezone(unittest.TestCase):

    def test_scheduler_invalid_timezone(self):
        """Test that invalid timezone in Scheduler raises a ValueError."""
        with self.assertRaises(ValueError):
            Scheduler(start_date="2025-12-25 10:00", time_zone="Invalid Timezone")

    def test_job_invalid_timezone(self):
        """Test that invalid timezone in job raises a ValueError."""
        scheduler = Scheduler(start_date="2025-12-25 10:00", time_zone=4)
        with self.assertRaises(ValueError):
            scheduler.job(start_date="2025-12-25 11:00", time_zone="Invalid")

    def test_job_no_timezone_inherits_scheduler(self):
        """Test that a job without a timezone inherits the Scheduler's timezone."""
        scheduler = Scheduler(start_date="2025-12-25 10:00", time_zone=5)
        job = scheduler.job(start_date="2025-12-25 11:00")
        self.assertEqual(job.job['time_zone'], scheduler.scheduler_timezone)

    def test_scheduler_and_job_default_to_current_timezone(self):
        """Test that both Scheduler and job default to the current system timezone when none is provided."""
        current_timezone = datetime.now().astimezone().tzinfo
        scheduler = Scheduler(start_date="2025-12-25 10:00")
        self.assertEqual( scheduler.scheduler_timezone, current_timezone)

        job = scheduler.job(start_date="2025-12-25 11:00")
        self.assertEqual(job.job['time_zone'], current_timezone)


if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(TestSchedulerTimezone)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_job_invalid_timezone (__main__.TestSchedulerTimezone.test_job_invalid_timezone)
Test that invalid timezone in job raises a ValueError. ... ok
test_job_no_timezone_inherits_scheduler (__main__.TestSchedulerTimezone.test_job_no_timezone_inherits_scheduler)
Test that a job without a timezone inherits the Scheduler's timezone. ... ok
test_scheduler_and_job_default_to_current_timezone (__main__.TestSchedulerTimezone.test_scheduler_and_job_default_to_current_timezone)
Test that both Scheduler and job default to the current system timezone when none is provided. ... ok
test_scheduler_invalid_timezone (__main__.TestSchedulerTimezone.test_scheduler_invalid_timezone)
Test that invalid timezone in Scheduler raises a ValueError. ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.010s

OK


### Startdate Testing All

In [6]:
class TestSchedulerStartDate(unittest.TestCase):

    def test_scheduler_invalid_stdate(self):
        """Test that invalid startdate in Scheduler raises a ValueError."""
        with self.assertRaises(ValueError):
            Scheduler(start_date="Something That doesn't belong here", time_zone=3)

    def test_job_invalid_stdate(self):
        """Test that invalid startdate in job raises a ValueError."""
        scheduler = Scheduler(start_date="2025-12-25 11:00", time_zone=4)
        with self.assertRaises(ValueError):
            scheduler.job(start_date="Nothing", time_zone=5)

    def test_job_no_stdate_inherits_scheduler(self):
        """Test that a job without a startdate inherits the Scheduler's startdate."""
        scheduler = Scheduler(start_date="2025-12-25 10:00", time_zone=5)
        job = scheduler.job(time_zone=5)
        self.assertEqual(job.job['startdate'].strftime('%Y-%m-%d %H:%M'), scheduler.scheduler_startdate.strftime('%Y-%m-%d %H:%M'))

    def test_scheduler_and_job_default_to_current_stdate(self):
        """Test that both Scheduler and job default to the current system startdate when none is provided."""
        current_time = datetime.now()
        scheduler = Scheduler(time_zone=4)
        self.assertEqual(scheduler.scheduler_startdate.strftime('%Y-%m-%d %H:%M'), current_time.strftime('%Y-%m-%d %H:%M'))

        job = scheduler.job(time_zone=4)
        self.assertEqual(job.job['startdate'].strftime('%Y-%m-%d %H:%M'), current_time.strftime('%Y-%m-%d %H:%M'))


    # def test_scheduler_waits_for_startdate(self):
    #     """Test that the job waits for the specified start date."""
    #     # Set a start date 5 seconds from now
    #     start_time = datetime.now() + timedelta(seconds=5)
    #     print(start_time)
    #     scheduler = Scheduler()
    #     job = scheduler.job(start_date=start_time.strftime('%Y-%m-%d %H:%M'), time_zone=4).repeat(3)
    #     
    #     start_wait_time = datetime.now()
    #     job.run()
    #     end_wait_time = datetime.now()
    #     
    #     # Ensure the end time is at least the start time
    #     self.assertGreaterEqual(end_wait_time, start_time, "Job did not wait until the start date.")
    #     self.assertGreaterEqual((end_wait_time - start_wait_time).total_seconds(), 5, "Job did not wait long enough.") #isn't properly written needs rewritting 

        
        

if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(TestSchedulerStartDate)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_job_invalid_stdate (__main__.TestSchedulerStartDate.test_job_invalid_stdate)
Test that invalid startdate in job raises a ValueError. ... ok
test_job_no_stdate_inherits_scheduler (__main__.TestSchedulerStartDate.test_job_no_stdate_inherits_scheduler)
Test that a job without a startdate inherits the Scheduler's startdate. ... ok
test_scheduler_and_job_default_to_current_stdate (__main__.TestSchedulerStartDate.test_scheduler_and_job_default_to_current_stdate)
Test that both Scheduler and job default to the current system startdate when none is provided. ... ok
test_scheduler_invalid_stdate (__main__.TestSchedulerStartDate.test_scheduler_invalid_stdate)
Test that invalid startdate in Scheduler raises a ValueError. ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.008s

OK


### EndDate Testing

In [7]:
class TestSchedulerUntil(unittest.TestCase):
    def test_job_end_date(self):
        """Test setting an end date for a job."""
        timezonesc = 4
        timezonejob = 3
        local_timezone = datetime.now().astimezone().tzinfo
    
        # Get current date and time
        current_time = datetime.now()
        
        # Add 2 minutes
        enddatejob = current_time + timedelta(minutes=2)
        enddatejob = datetime.strftime(enddatejob, "%Y-%m-%d %H:%M")

    
        scheduler = Scheduler(time_zone=timezonesc)
        job = scheduler.job(time_zone=timezonejob).until(enddatejob)
    
        if timezonejob is not None:
            original_timezone = timezone(timedelta(hours=timezonejob))
            if local_timezone != timezonejob:
                original_datetime = datetime.strptime(enddatejob, "%Y-%m-%d %H:%M")
                original_datetime = original_datetime.replace(tzinfo=original_timezone)
                comparison = original_datetime.astimezone(local_timezone)
                self.assertEqual(job.job['end_date'], comparison)
            else:
                self.assertEqual(job.job['end_date'], enddatejob)
        elif timezonejob is None and timezonesc is not None:
            original_timezone = timezone(timedelta(hours=timezonesc))
            if timezonesc != local_timezone:
                original_datetime = datetime.strptime(enddatejob, "%Y-%m-%d %H:%M")
                original_datetime = original_datetime.replace(tzinfo=original_timezone)
                comparison = original_datetime.astimezone(local_timezone)
                self.assertEqual(job.job['end_date'], comparison)
            else:
                self.assertEqual(job.job['end_date'], enddatejob)
        else:
            print('??????')

    


if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(TestSchedulerUntil)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_job_end_date (__main__.TestSchedulerUntil.test_job_end_date)
Test setting an end date for a job. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Threadng Test

In [8]:
class TestSchedulerThreading(unittest.TestCase):
    def test_run_all_with_threading(self):
        """Test running multiple jobs with threading enabled."""
        scheduler = Scheduler(threading=True,time_zone=0)

        # Create events to track the completion of tasks
        task1_event = Event()
        task2_event = Event()

        def task1():
            # print("Task 1 executed")
            time.sleep(2)  # Simulate a longer-running task
            task1_event.set()  # Mark task 1 as complete

        def task2():
            # print("Task 2 executed")
            task2_event.set()  # Mark task 2 as complete

        # Schedule multiple jobs
        scheduler.job( time_zone=4).do(task1, name="Task1").second().repeat(2)
        scheduler.job( time_zone=4).do(task2, name="Task2").second().repeat(2)

        # Run all jobs
        scheduler.run_all()

        # Wait for both tasks to complete
        self.assertTrue(task1_event.wait(timeout=5), "Task 1 did not complete in time")
        self.assertTrue(task2_event.wait(timeout=5), "Task 2 did not complete in time")

    def test_run_all_without_threading(self):
        """Test running multiple jobs without threading."""
        scheduler = Scheduler(threading=False, time_zone=0)

        task1_completed = False
        task2_completed = False

        def task1():
            nonlocal task1_completed
            # print("Task 1 executed")
            task1_completed = True

        def task2():
            nonlocal task2_completed
            # print("Task 2 executed")
            task2_completed = True

        # Schedule multiple jobs
        scheduler.job(time_zone=4).do(task1, name="Task1").second().repeat(2)
        scheduler.job( time_zone=4).do(task2, name="Task2").second().repeat(2)

        # Run all jobs
        scheduler.run_all()

        # Verify both tasks completed
        self.assertTrue(task1_completed, "Task 1 did not complete")
        self.assertTrue(task2_completed, "Task 2 did not complete")



if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(TestSchedulerThreading)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_run_all_with_threading (__main__.TestSchedulerThreading.test_run_all_with_threading)
Test running multiple jobs with threading enabled. ... ok
test_run_all_without_threading (__main__.TestSchedulerThreading.test_run_all_without_threading)
Test running multiple jobs without threading. ... ok

----------------------------------------------------------------------
Ran 2 tests in 6.638s

OK


## Units

### Interval and Unit Testing All

In [9]:
class TestSchedulerInterval(unittest.TestCase):

    def test_job_interval_second(self):
        """Test setting intervals for a job in seconds."""
        scheduler = Scheduler()
        job = scheduler.job().second(5)
        self.assertEqual(job.job['interval'], 5)
        self.assertEqual(job.job['unit'], 'second')

    def test_job_interval_minute(self):
        """Test setting intervals for a job in minutes."""
        scheduler = Scheduler()
        job = scheduler.job().minute(5)
        self.assertEqual(job.job['interval'], 5)
        self.assertEqual(job.job['unit'], 'minute')

    def test_job_interval_hour(self):
        """Test setting intervals for a job in hours."""
        scheduler = Scheduler()
        job = scheduler.job().hour(3)
        self.assertEqual(job.job['interval'], 3)
        self.assertEqual(job.job['unit'], 'hour')

    def test_job_interval_day(self):
        """Test setting intervals for a job in days."""
        scheduler = Scheduler()
        job = scheduler.job().day(4)
        self.assertEqual(job.job['interval'], 4)
        self.assertEqual(job.job['unit'], 'day')
        
    def test_job_interval_day_with_at(self):
        """Test 'hour' parameter for day method"""
        timezone_offset = 3
        local_timezone = datetime.now().astimezone().tzinfo
        original_timezone = timezone(timedelta(hours=timezone_offset))

        if local_timezone == original_timezone:
            runtime = '15:30'
            scheduler = Scheduler(time_zone=timezone_offset)
            job = scheduler.job(time_zone=timezone_offset).day(hour=runtime)
            self.assertEqual(job.job['at'], runtime)
        elif local_timezone != original_timezone:
            runtime = '15:30'
            scheduler = Scheduler(time_zone=timezone_offset)
            job = scheduler.job(time_zone=timezone_offset).day(hour=runtime)
            # Convert string time to a datetime object (with a fixed dummy date)
            runtime_dt = datetime.strptime(runtime, "%H:%M")
            runtime_dt = runtime_dt.replace(tzinfo=original_timezone)  # Apply original timezone
            # Convert to local timezone
            time_comparison = runtime_dt.astimezone(local_timezone)
            # Extract only the time portion to compare
            adjusted_runtime = time_comparison.strftime("%H:%M")
            self.assertEqual(job.job['at'], adjusted_runtime)

    def test_job_interval_week(self):
            """Test setting intervals for a job in weeks."""
            scheduler = Scheduler()
            job = scheduler.job().week(13)
            self.assertEqual(job.job['interval'], 13)
            self.assertEqual(job.job['unit'], 'week')

    def test_job_interval_month(self):
        """Test setting intervals for a job in months."""
        scheduler = Scheduler()
        job = scheduler.job().month(12)
        self.assertEqual(job.job['interval'], 12)
        self.assertEqual(job.job['unit'], 'month')

    def test_job_interval_year(self):
        """Test setting intervals for a job in years."""
        scheduler = Scheduler()
        job = scheduler.job().year(11)
        self.assertEqual(job.job['interval'], 11)
        self.assertEqual(job.job['unit'], 'year')

    def test_default_interval_second(self):
        """Test default interval for seconds."""
        scheduler = Scheduler()
        job = scheduler.job().second()  # No interval passed
        self.assertEqual(job.job['interval'], 1)
        self.assertEqual(job.job['unit'], 'second')

    def test_default_interval_minute(self):
        """Test default interval for minutes."""
        scheduler = Scheduler()
        job = scheduler.job().minute()  # No interval passed
        self.assertEqual(job.job['interval'], 1)
        self.assertEqual(job.job['unit'], 'minute')

    def test_default_interval_hour(self):
        """Test default interval for hours."""
        scheduler = Scheduler()
        job = scheduler.job().hour()  # No interval passed
        self.assertEqual(job.job['interval'], 1)
        self.assertEqual(job.job['unit'], 'hour')

    def test_default_interval_day(self):
        """Test default interval for days."""
        scheduler = Scheduler()
        job = scheduler.job(time_zone=3).day()  # No interval passed
        self.assertEqual(job.job['interval'], 1)
        self.assertEqual(job.job['unit'], 'day')

    def test_default_interval_week(self):
        """Test default interval for weeks."""
        scheduler = Scheduler()
        job = scheduler.job().week()  # No interval passed
        self.assertEqual(job.job['interval'], 1)
        self.assertEqual(job.job['unit'], 'week')

    def test_default_interval_month(self):
        """Test default interval for months."""
        scheduler = Scheduler()
        job = scheduler.job().month()  # No interval passed
        self.assertEqual(job.job['interval'], 1)
        self.assertEqual(job.job['unit'], 'month')

    def test_default_interval_year(self):
        """Test default interval for years."""
        scheduler = Scheduler()
        job = scheduler.job().year()  # No interval passed
        self.assertEqual(job.job['interval'], 1)
        self.assertEqual(job.job['unit'], 'year')
    
    def test_invalid_interval(self):
        """Test behavior for invalid interval values."""
        scheduler = Scheduler()
    
        # Test invalid minute intervals
        with self.assertRaises(ValueError):
            scheduler.job().minute(-1)  # Negative intervals are invalid
        with self.assertRaises(ValueError):
            scheduler.job().minute("invalid")  # Non-numeric intervals are invalid
    
        # Test invalid second intervals
        with self.assertRaises(ValueError):
            scheduler.job().second(-1)
        with self.assertRaises(ValueError):
            scheduler.job().second("invalid")
    
        # Test invalid hour intervals
        with self.assertRaises(ValueError):
            scheduler.job().hour(-1)
        with self.assertRaises(ValueError):
            scheduler.job().hour("invalid")
    
        # Test invalid day intervals
        with self.assertRaises(ValueError):
            scheduler.job().day(-1)
        with self.assertRaises(ValueError):
            scheduler.job().day("invalid")
    
        # Test invalid week intervals
        with self.assertRaises(ValueError):
            scheduler.job().week(-1)
        with self.assertRaises(ValueError):
            scheduler.job().week("invalid")
    
        # Test invalid month intervals
        with self.assertRaises(ValueError):
            scheduler.job().month(-1)
        with self.assertRaises(ValueError):
            scheduler.job().month("invalid")
    
        # Test invalid year intervals
        with self.assertRaises(ValueError):
            scheduler.job().year(-1)
        with self.assertRaises(ValueError):
            scheduler.job().year("invalid")



if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(TestSchedulerInterval)
    unittest.TextTestRunner(verbosity=2).run(suite)


test_default_interval_day (__main__.TestSchedulerInterval.test_default_interval_day)
Test default interval for days. ... ok
test_default_interval_hour (__main__.TestSchedulerInterval.test_default_interval_hour)
Test default interval for hours. ... ok
test_default_interval_minute (__main__.TestSchedulerInterval.test_default_interval_minute)
Test default interval for minutes. ... ok
test_default_interval_month (__main__.TestSchedulerInterval.test_default_interval_month)
Test default interval for months. ... ok
test_default_interval_second (__main__.TestSchedulerInterval.test_default_interval_second)
Test default interval for seconds. ... ok
test_default_interval_week (__main__.TestSchedulerInterval.test_default_interval_week)
Test default interval for weeks. ... ok
test_default_interval_year (__main__.TestSchedulerInterval.test_default_interval_year)
Test default interval for years. ... ok
test_invalid_interval (__main__.TestSchedulerInterval.test_invalid_interval)
Test behavior for inva

### Inner parameters of intervals

In [10]:
### დღის გაშვებაზე და მეტი რეინჯის სათებს რამე ერორი უნდა ედოთ? 

In [28]:
class TestSchedulerInnerParams(unittest.TestCase):


    def test_job_day_method_inner_params(self):
        """Test 'hour' parameter for day method"""
        timezone_offset = 3
        local_timezone = datetime.now().astimezone().tzinfo
        original_timezone = timezone(timedelta(hours=timezone_offset))
        
        # if code runs for local timezone
        if local_timezone == original_timezone:
            runtime = '15:30'
            scheduler = Scheduler(time_zone=timezone_offset)
            job = scheduler.job(time_zone=timezone_offset).day(hour=runtime)
            self.assertEqual(job.job['at'], runtime)

            # check if it handles not passed time for the day
            job2 = scheduler.job(time_zone=timezone_offset).day()
            time_str = job2.job['startdate'].strftime('%H:%M')
            self.assertEqual(job2.job['at'], time_str)

        # if code runs for different timezone
        elif local_timezone != original_timezone:
            runtime = '15:30'
            scheduler = Scheduler(time_zone=timezone_offset)
            
            job = scheduler.job(time_zone=timezone_offset).day(hour=runtime)
            runtime_dt = datetime.strptime(runtime, "%H:%M")   # Convert string time to a datetime object (with a fixed dummy date)
            runtime_dt = runtime_dt.replace(tzinfo=original_timezone)  # Apply original timezone
            time_comparison = runtime_dt.astimezone(local_timezone)  # Convert to local timezone
            # Extract only the time portion to compare
            adjusted_runtime = time_comparison.strftime("%H:%M")
            self.assertEqual(job.job['at'], adjusted_runtime)

            # check if it handles not passed time for the day
            job2 = scheduler.job(time_zone=timezone_offset).day()
            time_str = job2.job['startdate'].strftime('%H:%M')
            self.assertEqual(job2.job['at'], time_str)


    def test_job_week_method_inner_params_one(self):
            """Test 'weekday'(only one) and 'hour' parameters for week method"""
            timezone_offset = 3
            local_timezone = datetime.now().astimezone().tzinfo
            original_timezone = timezone(timedelta(hours=timezone_offset))
            
            # if code runs for local timezone
            if local_timezone == original_timezone:
                runday = [3]
                runtime = '15:30'
                
                # weekday #
                scheduler = Scheduler(time_zone=timezone_offset)
                job = scheduler.job(time_zone=timezone_offset).week(week_day=runday)
                self.assertEqual(job.job['kwargs']['week_day'], runday)
                
                # handling nan values for weekday if it takes from startdate
                job2 = scheduler.job(time_zone=timezone_offset, start_date='2025-04-07 10:00').week()
                self.assertEqual(job2.job['kwargs']['week_day'], [job2.job['startdate'].weekday()])

                # time #
                job3 = scheduler.job(time_zone=timezone_offset).week(hour=runtime)
                self.assertEqual(job3.job['at'], runtime)

                # handling nan values for time if it takes from startdate
                job4 = scheduler.job(time_zone=timezone_offset).week()
                time_str = job4.job['startdate'].strftime('%H:%M')
                self.assertEqual(job4.job['at'], time_str)

            # if code runs for different timezone
            elif local_timezone != original_timezone:
                runday = [6]
                runtime = '15:30'
                
                # weekday #
                scheduler = Scheduler(time_zone=timezone_offset)
                job = scheduler.job(time_zone=timezone_offset).week(week_day=runday)
                self.assertEqual(job.job['kwargs']['week_day'], runday)

                # handling nan values for weekday if it takes from startdate
                job2 = scheduler.job(time_zone=timezone_offset, start_date='2025-04-07 10:00').week()
                self.assertEqual(job2.job['kwargs']['week_day'], [job2.job['startdate'].weekday()])

                # time #
                job3 = scheduler.job(time_zone=timezone_offset).week(hour=runtime) 
                runtime_dt = datetime.strptime(runtime, "%H:%M")  # Convert string time to a datetime object (with a fixed dummy date)
                runtime_dt = runtime_dt.replace(tzinfo=original_timezone)  # Apply original timezone
                time_comparison = runtime_dt.astimezone(local_timezone) # Convert to local timezone
                adjusted_runtime = time_comparison.strftime("%H:%M")  # Extract only the time portion to compare
                self.assertEqual(job3.job['at'], adjusted_runtime)

                # check if it handles not passed time for the day
                job4 = scheduler.job(time_zone=timezone_offset).day()
                time_str = job4.job['startdate'].strftime('%H:%M')
                self.assertEqual(job4.job['at'], time_str)                
                


    def test_job_month_method_inner_params_one(self):
            """Test 'day'(only one which is date of the month) and 'hour' parameters for month method"""
            timezone_offset = 3
            local_timezone = datetime.now().astimezone().tzinfo
            original_timezone = timezone(timedelta(hours=timezone_offset))

            # if code runs for local timezone
            if local_timezone == original_timezone:
                rundate = 4
                erroredrundate = 300
                runtime = '15:30'
                
                # date of run #
                # test passed day
                scheduler = Scheduler(time_zone=timezone_offset)
                job = scheduler.job(time_zone=timezone_offset).month(day=rundate)
                self.assertEqual(job.job['on_day'], rundate)

                # check if it handles not passed time for the date
                job2 = scheduler.job(time_zone=timezone_offset, start_date='2025-04-25 10:00').month()
                self.assertEqual(job2.job['on_day'], job2.job['startdate'].day)
                
                # test raise of error for the date range 
                if erroredrundate<1 or erroredrundate>31:
                    with self.assertRaises(ValueError):
                        scheduler = Scheduler(time_zone=timezone_offset)
                        scheduler.job(time_zone=timezone_offset).month(day=erroredrundate)

                # time of run #
                job3 = scheduler.job(time_zone=timezone_offset).month(hour=runtime)
                self.assertEqual(job3.job['at'], runtime)

                # check if it handles not passed time for the time
                job4 = scheduler.job(time_zone=timezone_offset).month()
                time_str = job4.job['startdate'].strftime('%H:%M')
                self.assertEqual(job4.job['at'], time_str)

            # if code runs for different timezone
            elif local_timezone != original_timezone:
                rundate = 30
                runtime = '15:30'
                erroredrundate = 300


                # date of run #
                # test passed day
                scheduler = Scheduler(time_zone=timezone_offset)
                job = scheduler.job(time_zone=timezone_offset).month(day=rundate)
                self.assertEqual(job.job['on_day'], rundate)

                # check if it handles not passed time for the date
                job2 = scheduler.job(time_zone=timezone_offset, start_date='2025-04-25 10:00').month()
                self.assertEqual(job2.job['on_day'], job2.job['startdate'].day)

                # test raise of error for the date range 
                if erroredrundate<1 or erroredrundate>31:
                    with self.assertRaises(ValueError):
                        scheduler.job(time_zone=timezone_offset).month(day=erroredrundate)


                # time of run #
                job3 = scheduler.job(time_zone=timezone_offset).month(hour=runtime)
                runtime_dt = datetime.strptime(runtime, "%H:%M")  # Convert string time to a datetime object (with a fixed dummy date)
                runtime_dt = runtime_dt.replace(tzinfo=original_timezone)  # Apply original timezone
                time_comparison = runtime_dt.astimezone(local_timezone) # Convert to local timezone
                adjusted_runtime = time_comparison.strftime("%H:%M")  # Extract only the time portion to compare
                self.assertEqual(job3.job['at'], adjusted_runtime)

                # check if it handles not passed time for the day
                job4 = scheduler.job(time_zone=timezone_offset).day()
                time_str = job4.job['startdate'].strftime('%H:%M')
                self.assertEqual(job4.job['at'], time_str)





if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(TestSchedulerInnerParams)
    unittest.TextTestRunner(verbosity=2).run(suite)


test_job_day_method_inner_params (__main__.TestSchedulerInnerParams.test_job_day_method_inner_params)
Test 'hour' parameter for day method ... ok
test_job_month_method_inner_params_one (__main__.TestSchedulerInnerParams.test_job_month_method_inner_params_one)
Test 'day'(only one which is date of the month) and 'hour' parameters for month method ... ok
test_job_week_method_inner_params_one (__main__.TestSchedulerInnerParams.test_job_week_method_inner_params_one)
Test 'weekday'(only one) and 'hour' parameters for week method ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.007s

OK


16:30
18:58


### Calculate Next Run Testing All

In [12]:
class TestSchedulerCalculateNextrun(unittest.TestCase):

    def test_calculate_next_run_minute(self):
        """Test calculation of the next run time for a job with minute interval."""
        timezone_offset = 3
        local_timezone = datetime.now().astimezone().tzinfo
        original_timezone = timezone(timedelta(hours=timezone_offset))


        if local_timezone == original_timezone:
            scheduler = Scheduler(time_zone=timezone_offset)
            job = scheduler.job(time_zone=timezone_offset).minute(1)
            now = datetime.now()
            next_run = job.calculate_next_run(now)
            expected_next_run = (now + timedelta(minutes=1)).replace(second=0, microsecond=0)
            print(now, next_run)
            self.assertEqual(next_run, expected_next_run)
        elif local_timezone != original_timezone:
            scheduler = Scheduler(time_zone=timezone_offset)
            job = scheduler.job(time_zone=timezone_offset).minute(1)
            now = datetime.now()
            now_for_comparison = now.replace(tzinfo=original_timezone, microsecond=0)
            now_for_comparison = now_for_comparison.astimezone(local_timezone)
            next_run = job.calculate_next_run(now)
            expected_next_run = (now_for_comparison + timedelta(minutes=1)).replace(second=0, microsecond=0)
            # print(f'this is now:{now},\nthis is returned next run:{next_run},\nthis i what I compare it:{expected_next_run}')
            self.assertEqual(next_run, expected_next_run)
            

    def test_calculate_next_run_second(self):
        """Test calculation of the next run time for a job with second interval."""

        timezone_offset = 3
        local_timezone = datetime.now().astimezone().tzinfo
        original_timezone = timezone(timedelta(hours=timezone_offset))

        if local_timezone == original_timezone:
            scheduler = Scheduler(time_zone=timezone_offset)
            job = scheduler.job(time_zone=timezone_offset).second(1)
            now = datetime.now()
            next_run = job.calculate_next_run(now)
            expected_next_run = (now + timedelta(seconds=1)).replace(microsecond=0)
            print(now, next_run)
            self.assertEqual(next_run, expected_next_run)
        elif local_timezone != original_timezone:
            scheduler = Scheduler(time_zone=timezone_offset)
            job = scheduler.job(time_zone=timezone_offset).second(1)
            now = datetime.now()
            now_for_comparison = now.replace(tzinfo=original_timezone, microsecond=0)
            now_for_comparison = now_for_comparison.astimezone(local_timezone)
            next_run = job.calculate_next_run(now)
            expected_next_run = (now_for_comparison + timedelta(seconds=1)).replace(microsecond=0)
            # print(f'this is now:{now},\nthis is returned next run:{next_run},\nthis i what I compare it:{expected_next_run}')
            self.assertEqual(next_run, expected_next_run)
        
        
    def test_calculate_next_run_hour(self):
        """Test calculation of the next run time for a job with hour interval."""
        
        timezone_offset = 4
        local_timezone = datetime.now().astimezone().tzinfo
        original_timezone = timezone(timedelta(hours=timezone_offset))

        if local_timezone == original_timezone:
            scheduler = Scheduler(time_zone=timezone_offset)
            job = scheduler.job(time_zone=timezone_offset).hour(1)
            now = datetime.now()
            next_run = job.calculate_next_run(now)
            expected_next_run = (now + timedelta(hours=1)).replace(second=0,microsecond=0, tzinfo=original_timezone)
            self.assertEqual(next_run, expected_next_run)
        elif local_timezone != original_timezone:
            scheduler = Scheduler(time_zone=timezone_offset)
            job = scheduler.job(time_zone=timezone_offset).hour(1)
            now = datetime.now()
            now_for_comparison = now.replace(tzinfo=original_timezone, microsecond=0)
            now_for_comparison = now_for_comparison.astimezone(local_timezone)
            next_run = job.calculate_next_run(now)
            expected_next_run = (now_for_comparison + timedelta(hours=1)).replace(second=0,microsecond=0)
            self.assertEqual(next_run, expected_next_run)

    def test_calculate_next_run_day(self):
        """Test calculation of the next run time for a job with day interval."""

        timezone_offset = 4
        local_timezone = datetime.now().astimezone().tzinfo
        original_timezone = timezone(timedelta(hours=timezone_offset))
        # Given date string
        startdate_str = "2025-02-22 15:00"
        # Convert to datetime object
        startdate_dt = datetime.strptime(startdate_str, "%Y-%m-%d %H:%M")

        if local_timezone == original_timezone:
            scheduler = Scheduler(time_zone=timezone_offset)
            job = scheduler.job(time_zone=timezone_offset,start_date=startdate_str).day(1)
            now = datetime.now()
            next_run = job.calculate_next_run(now)
            expected_next_run = (now + timedelta(days=1)).replace(hour=0, minute=0, second=0,microsecond=0, tzinfo=original_timezone)
            print(next_run,expected_next_run)
            self.assertEqual(next_run, expected_next_run)
        elif local_timezone != original_timezone:
            scheduler = Scheduler(time_zone=timezone_offset)
            job = scheduler.job(time_zone=timezone_offset).day(1)
            now = datetime.now()
            now_for_comparison = now.replace(tzinfo=original_timezone, microsecond=0)
            now_for_comparison = now_for_comparison.astimezone(local_timezone)
            next_run = job.calculate_next_run(now)
            expected_next_run = (now_for_comparison + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
            print(next_run,expected_next_run )
            self.assertEqual(next_run, expected_next_run)

    # def test_calculate_next_run_week(self):
    #     """Test calculation of the next run time for a job with week interval."""
    # 
    #     timezone_offset = 4
    #     local_timezone = datetime.now().astimezone().tzinfo
    #     original_timezone = timezone(timedelta(hours=timezone_offset))
    #     # Given date string
    #     startdate_str = "2025-02-22 15:00"
    #     # Convert to datetime object
    #     startdate_dt = datetime.strptime(startdate_str, "%Y-%m-%d %H:%M")
    # 
    #     if local_timezone == original_timezone:
    #         scheduler = Scheduler(time_zone=timezone_offset)
    #         job = scheduler.job(time_zone=timezone_offset,start_date=startdate_str).day(1)
    #         now = datetime.now()
    #         next_run = job.calculate_next_run(now)
    #         expected_next_run = (now + timedelta(days=1)).replace(hour=0, minute=0, second=0,microsecond=0, tzinfo=original_timezone)
    #         print(next_run,expected_next_run)
    #         self.assertEqual(next_run, expected_next_run)
    #     elif local_timezone != original_timezone:
    #         scheduler = Scheduler(time_zone=timezone_offset)
    #         job = scheduler.job(time_zone=timezone_offset).day(1)
    #         now = datetime.now()
    #         now_for_comparison = now.replace(tzinfo=original_timezone, microsecond=0)
    #         now_for_comparison = now_for_comparison.astimezone(local_timezone)
    #         next_run = job.calculate_next_run(now)
    #         expected_next_run = (now_for_comparison + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
    #         print(next_run,expected_next_run )
    #         self.assertEqual(next_run, expected_next_run)
    
        
        
    # def test_calculate_next_run_week(self):
    #     """Test calculation of the next run time for a job with week interval."""
    #     scheduler = Scheduler(time_zone=4)
    #     job = scheduler.job(time_zone=4).week(1)
    #     now = datetime.now()
    #     next_run = job.calculate_next_run(now)
    #     expected_next_run = (now + timedelta(weeks=1)).replace(hour=0, minute=0, second=0, microsecond=0)
    #     self.assertEqual(next_run, expected_next_run)
    # 
    # def test_calculate_next_run_month(self):
    #     """Test calculation of the next run time for a job with month interval."""
    #     scheduler = Scheduler(time_zone=4)
    #     job = scheduler.job(time_zone=4).month(1)
    #     now = datetime.now()
    #     next_month = (now.month % 12) + 1
    #     next_year = now.year + (now.month // 12)
    #     next_run = job.calculate_next_run(now)
    #     expected_next_run = now.replace(year=next_year, month=next_month, day=1, hour=0, minute=0, second=0, microsecond=0)
    #     self.assertEqual(next_run, expected_next_run)
    # 
    # def test_calculate_next_run_year(self):
    #     """Test calculation of the next run time for a job with year interval."""
    #     scheduler = Scheduler(time_zone=4)
    #     job = scheduler.job(time_zone=4).year(1)
    #     now = datetime.now()
    #     next_run = job.calculate_next_run(now)
    #     expected_next_run = now.replace(year=now.year + 1, month=1, day=1, hour=0, minute=0, second=0, microsecond=0)
    #     self.assertEqual(next_run, expected_next_run)
    # 


if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(TestSchedulerCalculateNextrun)
    unittest.TextTestRunner(verbosity=2).run(suite)


test_calculate_next_run_day (__main__.TestSchedulerCalculateNextrun.test_calculate_next_run_day)
Test calculation of the next run time for a job with day interval. ... ERROR
test_calculate_next_run_hour (__main__.TestSchedulerCalculateNextrun.test_calculate_next_run_hour)
Test calculation of the next run time for a job with hour interval. ... ok
test_calculate_next_run_minute (__main__.TestSchedulerCalculateNextrun.test_calculate_next_run_minute)
Test calculation of the next run time for a job with minute interval. ... ok
test_calculate_next_run_second (__main__.TestSchedulerCalculateNextrun.test_calculate_next_run_second)
Test calculation of the next run time for a job with second interval. ... ok

ERROR: test_calculate_next_run_day (__main__.TestSchedulerCalculateNextrun.test_calculate_next_run_day)
Test calculation of the next run time for a job with day interval.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Use

## Repeat Testing All

In [13]:
class TestSchedulerRepeat(unittest.TestCase):

    def test_job_repeat(self):
        """Test that a job can be set to repeat a specified number of times."""
        scheduler = Scheduler(start_date="2025-12-25 10:00", time_zone=5)
        job = scheduler.job(start_date="2025-12-25 11:00", time_zone=5).repeat(5)
        self.assertEqual(job.job['repeats'], 5)


        # self.assertEqual(job.job['repeats'], )

    # def test_job_runs_until_end_date(self):
    #     # Mock the run logic to count job executions
    #     execution_count = 0
    # 
    #     def mock_run_job():
    #         nonlocal execution_count
    #         execution_count += 1
    #         print(f'this is execution count {execution_count}')
    #     """Test that a job keeps running until the 'until' end date, even if the repeat count is reached."""
    #     scheduler = Scheduler(start_date="2025-01-13 21:15", time_zone=4)
    #     # end_date = "2025-01-13 15:31"
    #     job = scheduler.job(start_date="2025-01-13 15:37", time_zone=4).do(mock_run_job, name='something').second().repeat(3)
    #     scheduler.run_all()
    # 
    #     # Check that the job has an end_date set
    #     # self.assertIsNotNone(job.job['end_date'])
    #     
    #     # Check that the job repeated exactly 2 times
    #     self.assertGreater(execution_count, 2)
        
        # Ensure the job stopped at or before the 'until' end date
        # self.assertGreaterEqual(job.job['end_date'], job.job['startdate'])
        # Convert the end date to a datetime object for comparison
        # expected_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M')
        # actual_end_date =job.job['end_date']
    
        # self.assertLessEqual(actual_end_date, expected_end_date)


if __name__ == "__main__":
    suite = unittest.TestLoader().loadTestsFromTestCase(TestSchedulerRepeat)
    unittest.TextTestRunner(verbosity=2).run(suite)

test_job_repeat (__main__.TestSchedulerRepeat.test_job_repeat)
Test that a job can be set to repeat a specified number of times. ... ok

----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


In [14]:


    # 
    # def test_job_second_interval(self):
    #     """
    #     Test creating a job that runs every X seconds.
    #     We verify that the unit and interval are correctly set 
    #     and that the next run is calculated properly.
    #     """
    #     scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
    #     job = (
    #         scheduler
    #         .job(start_date="2024-12-25 10:05", time_zone=0)
    #         .second(5)  # set job to run every 5 seconds
    #     )
    # 
    #     self.assertEqual(job.job['unit'], 'second')
    #     self.assertEqual(job.job['interval'], 5)
    # 
    #     # Check next_run calculation
    #     now = datetime(2024, 12, 25, 10, 5, 0)
    #     next_run = job.calculate_next_run(now)
    #     expected_next_run = now + timedelta(seconds=5)
    #     self.assertEqual(next_run, expected_next_run)
    # 
    # def test_job_minute_interval(self):
    #     """
    #     Test creating a job that runs every X minutes.
    #     We verify that the unit and interval are correctly set 
    #     and that the next run is calculated properly.
    #     """
    #     scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=0)
    #     job = (
    #         scheduler
    #         .job(start_date="2024-12-25 10:30", time_zone=0)
    #         .minute(10)  # runs every 10 minutes
    #     )
    # 
    #     self.assertEqual(job.job['unit'], 'minute')
    #     self.assertEqual(job.job['interval'], 10)
    # 
    #     now = datetime(2024, 12, 25, 10, 30, 0)
    #     next_run = job.calculate_next_run(now)
    #     expected_next_run = now.replace(second=0) + timedelta(minutes=10)
    #     self.assertEqual(next_run, expected_next_run)
    # 
    # def test_job_hour_interval(self):
    #     """
    #     Test creating a job that runs every X hours.
    #     We verify that the unit and interval are correctly set 
    #     and that the next run is calculated properly.
    #     """
    #     scheduler = Scheduler(start_date="2024-12-25 08:00", time_zone=0)
    #     job = (
    #         scheduler
    #         .job(start_date="2024-12-25 09:00", time_zone=0)
    #         .hour(2)  # runs every 2 hours
    #     )
    # 
    #     self.assertEqual(job.job['unit'], 'hour')
    #     self.assertEqual(job.job['interval'], 2)
    # 
    #     now = datetime(2024, 12, 25, 9, 0, 0)
    #     next_run = job.calculate_next_run(now)
    #     # The method zeroes out minute/second, so we add 2 hours to the top of the hour
    #     expected_next_run = now.replace(minute=0, second=0) + timedelta(hours=2)
    #     self.assertEqual(next_run, expected_next_run)
    # 
    # def test_job_week_interval(self):
    #     """
    #     Test creating a job that runs every X weeks on certain weekdays.
    #     We verify that the unit, interval, and weekdays are set correctly 
    #     and that the next run is calculated properly.
    #     """
    #     scheduler = Scheduler(start_date="2024-12-20 09:00", time_zone=0)
    #     # Suppose we want it to run every 1 week on Mondays (weekday=0) at 10:00
    #     # 'week_day=0' -> Monday. If the test date is also Monday, 
    #     # we confirm that the next run date gets scheduled accordingly.
    #     job = (
    #         scheduler
    #         .job(start_date="2024-12-23 09:00", time_zone=0)  # 2024-12-23 is a Monday
    #         .week(every=1, week_day=0, hour="10:00")
    #     )
    # 
    #     self.assertEqual(job.job['unit'], 'week')
    #     self.assertEqual(job.job['interval'], 1)
    #     self.assertIn(0, job.job['week_day'])  # 0 means Monday
    #     self.assertEqual(job.job['at'], "10:00")
    # 
    #     # Let's check next_run calculation on 2024-12-23 at 09:00
    #     now = datetime(2024, 12, 23, 9, 0, 0)
    #     next_run = job.calculate_next_run(now)
    # 
    #     # Since 'hour="10:00"', the next run the same day (2024-12-23) at 10:00 
    #     # if 'calculate_next_run' logic treats "now" as 09:00.
    #     expected_next_run = now.replace(hour=10, minute=0, second=0)
    #     self.assertEqual(next_run, expected_next_run)

    # def test_job_week_multiple_days(self):
    #     """
    #     Test creating a job that runs every week on multiple weekdays.
    #     For example, Monday and Wednesday, verifying next_run is the earliest upcoming day.
    #     """
    #     scheduler = Scheduler(start_date="2024-12-20 09:00", time_zone=0)
    #     # Suppose we want it to run every week on Monday (0) and Wednesday (2), at 08:00
    #     job = (
    #         scheduler
    #         .job(start_date="2024-12-23 07:59", time_zone=0)  # 2024-12-23 is Monday
    #         .week(every=1, week_day=[0, 2], hour="08:00")
    #     )
    # 
    #     self.assertEqual(job.job['unit'], 'week')
    #     self.assertEqual(job.job['interval'], 1)
    #     self.assertIn(0, job.job['week_day'])  # Monday
    #     self.assertIn(2, job.job['week_day'])  # Wednesday
    #     self.assertEqual(job.job['at'], "08:00")
    # 
    #     # If current time is Monday 07:59, next_run should be Monday 08:00 (today)
    #     now = datetime(2024, 12, 23, 7, 59, 0)
    #     next_run = job.calculate_next_run(now)
    #     expected_next_run = now.replace(hour=8, minute=0, second=0)
    #     self.assertEqual(next_run, expected_next_run)

    # def test_job_with_timezones_second(self):
    #     """
    #     Test that a job set to run every X seconds honors different time zones
    #     by checking how the start date is converted to local time.
    #     """
    #     # Let's pick a scheduler in UTC+2
    #     scheduler = Scheduler(start_date="2024-12-25 10:00", time_zone=2)
    # 
    #     # Now create a job that starts in UTC-1
    #     job = scheduler.job(start_date="2024-12-25 10:00", time_zone=-1).second(15)
    # 
    #     self.assertEqual(job.job['unit'], 'second')
    #     self.assertEqual(job.job['interval'], 15)
    # 
    #     # Check that the job's start_date is converted properly to local time
    #     # according to the system's local timezone. 
    #     # (We won't do a hard-coded assert here because each environment can differ.
    #     #  Instead, we ensure it doesn't raise any exceptions and that the tzinfo is set.)
    #     self.assertIsNotNone(job.job['startdate'].tzinfo, "Job startdate timezone should not be None")
    # 
    #     # We can still do a sanity check: the job's 'time_zone' is UTC-1
    #     self.assertEqual(job.job['time_zone'].utcoffset(None), timedelta(hours=-1))
    # 
    #     # Confirm next run calculation from a reference now
    #     now_local = datetime(2024, 12, 25, 10, 0, 0)  # local time
    #     next_run = job.calculate_next_run(now_local)
    #     expected_next_run = now_local + timedelta(seconds=15)
    #     self.assertEqual(next_run, expected_next_run)

    # def test_job_with_timezones_hour(self):
    #     """
    #     Test that a job set to run every X hours works properly with different time zones.
    #     """
    #     # Scheduler in UTC+5
    #     scheduler = Scheduler(start_date="2024-12-25 00:00", time_zone=5)
    # 
    #     # Create a job that starts in UTC-3
    #     job = scheduler.job(start_date="2024-12-25 05:00", time_zone=-3).hour(1)
    # 
    #     self.assertEqual(job.job['unit'], 'hour')
    #     self.assertEqual(job.job['interval'], 1)
    # 
    #     # Check that the job's start_date is converted properly to local time
    #     self.assertIsNotNone(job.job['startdate'].tzinfo)
    # 
    #     # Confirm next run calculation from a reference now
    #     # We pretend "now" is local system time; we just want to see if
    #     # the job calculates the next run in 1 hour from that reference.
    #     now_local = datetime(2024, 12, 25, 5, 0, 0)
    #     next_run = job.calculate_next_run(now_local)
    #     expected_next_run = now_local.replace(minute=0, second=0) + timedelta(hours=1)
    #     self.assertEqual(next_run, expected_next_run)

    # def test_run_function_basic_execution(self):
    #     """
    #     Test that the 'run' method actually calls the assigned function
    #     after waiting until the start_date.
    #     """
    #     scheduler = Scheduler(threading=False, start_date="2099-01-01 00:00", time_zone=0)
    # 
    #     # We'll pick a start date slightly in the near future so we don't raise ValueError
    #     # (the code checks that start_date >= now).
    #     job_start_date_str = (datetime.utcnow() + timedelta(seconds=1)).strftime('%Y-%m-%d %H:%M')
    # 
    #     # A simple flag to see if our job function was called.
    #     job_executed = {'called': False}
    # 
    #     def dummy_task():
    #         job_executed['called'] = True
    # 
    #     job = scheduler.job(start_date=job_start_date_str, time_zone=0).do(dummy_task, name="BasicExecutionTest")
    # 
    #     # Run all jobs (non-threaded).
    #     # The job should sleep until the job start date, then run `dummy_task`.
    #     scheduler.run_all()
    # 
    #     self.assertTrue(
    #         job_executed['called'],
    #         msg="The job's function should have been called once after its start date."
    #     )
    # 
    # def test_run_function_does_not_run_before_start_date(self):
    #     """
    #     Test that if the current time is before the job's start date,
    #     the job waits, and then runs exactly once the time arrives.
    #     """
    #     scheduler = Scheduler(threading=False, start_date="2099-01-01 00:00", time_zone=0)
    # 
    #     # Start date is ~2 seconds from now
    #     future_time = datetime.utcnow() + timedelta(seconds=2)
    #     job_start_date_str = future_time.strftime('%Y-%m-%d %H:%M')  # e.g. '2099-01-01 10:30'
    # 
    #     job_executed = {'count': 0}
    # 
    #     def dummy_task():
    #         job_executed['count'] += 1
    # 
    #     job = scheduler.job(start_date=job_start_date_str, time_zone=0).do(dummy_task, name="WaitTest")
    # 
    #     start_test_time = datetime.utcnow()
    #     scheduler.run_all()  # This will block until the job can run
    #     end_test_time = datetime.utcnow()
    # 
    #     # The job should have run exactly once
    #     self.assertEqual(
    #         job_executed['count'],
    #         1,
    #         msg="Job function should execute exactly once."
    #     )
    #     # Check the difference in run time is at least ~2 seconds
    #     self.assertGreaterEqual(
    #         (end_test_time - start_test_time).total_seconds(),
    #         2,
    #         msg="Scheduler should have waited at least 2 seconds before running the job."
    #     )
    # 
    # def test_run_function_with_end_date(self):
    #     """
    #     Test that the job stops running after the end_date is reached.
    #     We create a job that would normally run multiple times, but has an end_date in the near future.
    #     """
    #     # We'll create a scheduler with a start date well in the future to avoid ValueError.
    #     scheduler = Scheduler(threading=False, start_date="2099-01-01 00:00", time_zone=0)
    # 
    #     # Start the job now, repeating every second, but set an end_date ~2 seconds from now.
    #     now_utc = datetime.utcnow()
    #     start_date_str = now_utc.strftime('%Y-%m-%d %H:%M')
    #     end_date_str = (now_utc + timedelta(seconds=3)).strftime('%Y-%m-%d %H:%M')
    # 
    #     job_run_count = {'count': 0}
    # 
    #     def dummy_repeating_task():
    #         job_run_count['count'] += 1
    # 
    #     job = (
    #         scheduler
    #         .job(start_date=start_date_str, time_zone=0)
    #         .second(1)            # run every 1 second
    #         .do(dummy_repeating_task, name="RepeatingJob")
    #         .until(end_date_str)  # end date is ~3 seconds out
    #     )
    # 
    #     # This will block and run the job until the end_date is reached
    #     scheduler.run_all()
    # 
    #     self.assertGreater(
    #         job_run_count['count'],
    #         1,
    #         msg="Job should run multiple times before the end date occurs."
    #     )
    # 
    #     # Should eventually stop when end_date is reached
    #     # We don't have a strict upper bound because of timing granularity, 
    #     # but we can at least confirm it didn't run indefinitely.
    # 
    # @patch('time.sleep', return_value=None)
    # @patch('datetime.datetime')
    # def test_run_function_with_mocked_time(self, mock_datetime, mock_sleep):
    #     """
    #     Demonstrate how to test 'run' more quickly by mocking out time.sleep and datetime.datetime.now.
    #     This test asserts that the job is executed after the 'start_date' once 'now' >= start_date.
    #     """
    #     # Create a fixed reference point for 'now'
    #     fixed_now = datetime(2099, 1, 1, 12, 0, 0, tzinfo=timezone.utc)
    # 
    #     # Mock datetime.datetime.now() to return fixed_now initially.
    #     mock_datetime.now.return_value = fixed_now
    # 
    #     # Set up the scheduler
    #     scheduler = Scheduler(threading=False, start_date="2099-01-01 11:59", time_zone=0)
    # 
    #     job_start_date_str = "2099-01-01 12:00"
    #     job_executed = {'called': False}
    # 
    #     def dummy_task():
    #         job_executed['called'] = True
    # 
    #     job = scheduler.job(start_date=job_start_date_str, time_zone=0).do(dummy_task, name="MockedTimeTest")
    # 
    #     # The first time run() checks 'now', it sees 12:00 == job start time => job can run immediately.
    #     scheduler.run_all()
    # 
    #     self.assertTrue(
    #         job_executed['called'],
    #         "Job should run after the mocked current time equals start date."
    #     )
    #     # Confirm that time.sleep was called at least once or zero times depending on how the code checks the equality
    #     # In many cases, it won't sleep at all if now == start_date. 
    #     mock_sleep.assert_called()  # or assert_not_called() if code never calls sleep when times are equal
    # 
    # def test_run_function_timezone_conversion(self):
    #     """
    #     Test that the job respects time zones when deciding if 'now' < job's start date.
    #     We create a scheduler in UTC+2, and the job in UTC-1, then see that the job 
    #     won't run until local time matches that start date.
    #     """
    #     # We'll choose a scheduler start_date in UTC+2 that is "in the future"
    #     # to avoid the ValueError for the scheduler itself.
    #     scheduler = Scheduler(threading=False, start_date="2099-01-02 00:00", time_zone=2)
    # 
    #     # Let's say the job wants to start at "2099-01-02 00:00" in UTC-1
    #     job_start_date_str = "2099-01-02 00:00"
    #     job_executed = {'called': False}
    # 
    #     def dummy_task():
    #         job_executed['called'] = True
    # 
    #     # Create the job
    #     job = scheduler.job(start_date=job_start_date_str, time_zone=-1).do(dummy_task, name="TZTest")
    # 
    #     # Because the code converts job's start date from UTC-1 to local tz 
    #     # (which might be different from your system's tz), 
    #     # the actual sleep time could vary. We'll rely on the code's 
    #     # built-in sleep rather than mocking. 
    #     scheduler.run_all()
    # 
    #     # Since we picked a date well in the future (2099-01-02),
    #     # this test won't truly run unless your system clock is also far in the future. 
    #     # Typically, you'd mock datetime.now() or time.sleep() to simulate that point in time.
    #     # We'll do a simple assertion that the code didn't crash
    #     # and that the job has a valid tzinfo set.
    #     self.assertIsNotNone(job.job['startdate'].tzinfo, "Job startdate should have a tzinfo.")
    #     self.assertFalse(
    #         job_executed['called'],
    #         msg="Without mocking time or advancing system clock, the job likely won't have run yet."
    #     )

        # In a real test environment, you'd mock or freeze time, so that 
        # the job 'thinks' it's already 2099-01-02 00:00 in UTC-1, 
        # and then you'd verify `job_executed['called'] == True`.

if __name__ == "__main__":
    unittest.main(argv=[''], exit=False, verbosity=2)


    # timezonesc = 5
    # timezonejob = 0
    # start_datesc = "2024-12-25 10:00"
    # startdatejob = "2024-12-25 11:00"
    # 
    # 
    # 
    # scheduler = Scheduler(start_date=start_datesc, time_zone=timezonesc)
    # job_to_run = scheduler.job(start_date=startdatejob, time_zone=timezonejob)
    # 
    # 
    # local_timezone = datetime.now().astimezone().tzinfo
    # # Parse the original date
    # original_datetime = datetime.strptime(startdatejob, "%Y-%m-%d %H:%M")
    
    # Add the original timezone dynamically
    # original_timezone = timezone(timedelta(hours=tmzone))

    # if timezonejob is not None:
    #     original_timezone = timezone(timedelta(hours=timezonejob))
    #     original_datetime = original_datetime.replace(tzinfo=original_timezone)
    # 
    #     if local_timezone != original_timezone:
    #         start_date = datetime.strptime(startdatejob, '%Y-%m-%d %H:%M')
    #         comparison = start_date.astimezone(local_timezone)
    #         # self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), comparison)
    #         print(f"whhhhhaaaatattttt{comparison, job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M')}")
    #     else:
    #         # self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), startdatejob)
    #         print('fwvvv')
    # elif timezonejob is None and timezonesc is not None:
    #     if timezonesc != local_timezone:
    #         start_date = datetime.strptime(startdatejob, '%Y-%m-%d %H:%M')
    #         comparison = start_date.astimezone(local_timezone)
    #         # self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), comparison)
    #         print(f'idk{comparison}')
    #     else:
    #         # self.assertEqual(job_to_run.job['startdate'].strftime('%Y-%m-%d %H:%M'), startdatejob)
    #         print('fsvvfvfvsfv')
    # else:
    #     print('wazzaaaa')

IndentationError: expected an indented block after 'if' statement on line 353 (53389963.py, line 354)